# Visualization of Covid-19 cases by Zipcodes with Python/Bokeh

### Import the required modules

In [1]:
import pandas as pd
import numpy as np
import holoviews as hv

import tabula
import requests
import datetime
import geopandas as gpd
import pandas as pd
import json
from dateutil import parser

In [2]:
import tabula
import requests
import datetime
import geopandas as gpd
import pandas as pd
import json
from dateutil import parser
from bokeh.io import reset_output, output_notebook, show

from bokeh.plotting import figure
from bokeh.models import Div, Column, Row


#make bokeh output to notebook
reset_output()
output_notebook()

Loading BokehJS ...

## import data

In [7]:
df = pd.read_json('all_dates_count_df.json',convert_dates=['ReportedDate', 'UpdatedDatetime'])


In [16]:
df = df[['ZipCode', 'CaseCount', 'ReportedDate']]

In [100]:
tmp = df.groupby('ZipCode')['CaseCount'].max()>50
selected_zipcodes = tmp[tmp==True].index
print(selected_zipcodes)
df = df[df['ZipCode'].isin(selected_zipcodes)]
df

Index(['91910', '91911', '91913', '91915', '91942', '91950', '91977', '92019',
       '92020', '92021', '92025', '92101', '92102', '92103', '92104', '92105',
       '92113', '92114', '92115', '92126', '92139', '92154', '92173',
       'Unknown***'],
      dtype='object', name='ZipCode')


,ZipCode,CaseCount,ReportedDate
0,91911,202,2020-05-01
2,92021,120,2020-05-01
4,92025,58,2020-05-01
13,92019,58,2020-05-01
24,92020,115,2020-05-01
...,...,...,...
1218,91910,68,2020-04-17
1239,92025,26,2020-04-17
1241,92021,70,2020-04-17
1242,92020,71,2020-04-17


In [98]:
from bokeh.models import ColumnDataSource
source = ColumnDataSource(df)
source.data.keys()

dict_keys(['index', 'ZipCode', 'CaseCount', 'ReportedDate'])

In [103]:
from bokeh.models import CheckboxButtonGroup, RadioButtonGroup
from bokeh.layouts import Row, Column
from bokeh.models import Plot, Label, CDSView, Filter, Slider, CustomJS, CustomJSFilter

zipcodes = list(df['ZipCode'].unique())

choice = RadioButtonGroup(
        labels= zipcodes, active=0)

text = Div(text="""<font size="4">""" + 'Case Count at ' + str(zipcodes[0]) + """</font>""")

# This callback is crucial, otherwise the filter will not be triggered when the slider changes
callback = CustomJS(args=dict(source=source,zipcodes = zipcodes,text = text,choice = choice),
                    code="""
                 text.text = "<font size=4> Case Count at " +zipcodes[choice.active]+ "</font>";
                source.change.emit();
""")

choice.js_on_change('active', callback)

# Define the custom filter to return the indices from 0 to the desired percentage of total data rows. You could also compare against values in source.data
js_filter = CustomJSFilter(args=dict(choice=choice, zipcodes=zipcodes),
                           code='''
            var indices = [];
            for (var i = 0; i <= source.data['ZipCode'].length; i++){
                if (source.data['ZipCode'][i] == zipcodes[choice.active]) {
                    indices.push(i)
                }
            }
            return indices
''')

# Use the filter in a view

long_view = CDSView(source=source, filters=[js_filter])
p = figure(x_axis_type ='datetime', plot_height=250, title="CaseCounts",
           toolbar_location=None, tools="")

p.step(x='ReportedDate', y='CaseCount', source = source, view = long_view, width=0.9)

show(Column(choice, text, p))